In [2]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score
from plotly.subplots import make_subplots
import statsmodels.api as sm

Using TensorFlow backend.


In [11]:
data = pd.read_csv("hourlySummaries.csv")
data["aaveHourlyPctChange"] = data["aaveHourlyChange"] / data["aavePriceUSD"]
data = data[data.columns[1:]]
data = data.loc[:"2022-01-06 00:00:00"]

train_weight = 0.80 # Using 80% of data for training, 20% for testing
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [12]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]

y_train = df_train["aaveHourlyPctChange"]
y_test = df_test["aaveHourlyPctChange"]

In [13]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [14]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [15]:
adf, p, usedlag, nobs, cvs,aic=sm.tsa.stattools.adfuller(y_train)

In [16]:
# Null hypothesis: Non Stationarity exists in the series.
# Alternative Hypothesis: Stationarity exists in the series

print("Critical Value: ", adf)
print("p-value" , p)
print("T-values for adfuller: ", cvs)
# Reject the null -- stationarity exists

Critical Value:  -27.30622331262927
p-value 0.0
T-values for adfuller:  {'1%': -3.434403043188966, '5%': -2.863330182785859, '10%': -2.567723055843177}


In [18]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_train.index, y=df_test.aaveHourlyChange,
                    mode='lines',
                    name='Aave Change'),
                    secondary_y=True)

Data is stationary 
-27.30 > -3.43 

# Alpha RNN

In [20]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [21]:
max_epoches=1000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

In [22]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)


2022-03-23 20:59:38.408605: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 20:59:38.410515: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/1000
1618/1618 [==============================] - 6s 3ms/step - loss: 0.1487
Epoch 2/1000
1618/1618 [==============================] - 0s 204us/step - loss: 0.1322
Epoch 3/1000
1618/1618 [==============================] - 0s 150us/step - loss: 0.1148
Epoch 4/1000
1618/1618 [==============================] - 0s 130us/step - loss: 0.0981
Epoch 5/1000
1618/1618 [==============================] - 0s 95us/step - loss: 0.0825
Epoch 6/1000
1618/1618 [==============================] - 0s 89us/step - loss: 0.0682
Epoch 7/1000
1618/1618 [==============================] - 0s 100us/step - loss: 0.0555
Epoch 8/1000
1618/1618 [==============================] - 0s 88us/step - loss: 0.0444
Epoch 9/1000
1618/1618 [==============================] - 0s 83us/step - loss: 0.0345
Epoch 10/1000
1618/1618 [==============================] - 0s 78us/step - loss: 0.0265
Epoch 11/1000
1618/1618 [==============================] - 0s 82us/step - loss: 0.0206
Epoch 12/1000
1618/1618 [========================

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_1 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [24]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred, multioutput='variance_weighted')

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

1 - (1-r2_test)*(len(y_test_reg)-1)/(len(y_test_reg)-x_test_reg.shape[1]-1)

1618/1618 [==============================] - 2s 1ms/step
Train MSE:  0.00026901351968129373
Train r2:  -0.1350844005049061
405/405 [==============================] - 0s 210us/step
Test MSE:  0.00023756455446784233
Test r2:  -0.2757342492741697


-0.38176042012537414

In [27]:
df_test.aavePriceUSD.iloc[0]

488.194300113

In [35]:
pred_prices = []
for i in range(1,len(df_test)):
    price = df_test.aavePriceUSD.iloc[i-1] * (1 + y_pred[i])
    pred_prices.append(price[0])

In [36]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_test.index, y=pred_prices,
                    mode='lines',
                    name='Aave Predicted'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.aavePriceUSD,
                    mode='lines',
                    name='Aave Actual'),
                    secondary_y=True)

# Using changes in price to predict numBorrows

In [ ]:
# X_train = df_train["aaveHourlyChange"]
# X_test = df_test["aaveHourlyChange"]

# y_train = df_train[["numBorrows"]]
# y_test = df_test[["numBorrows"]]

In [ ]:
# x_train_reg = np.array(X_train)
# x_test_reg = np.array(X_test)
# y_train_reg = np.array(y_train)
# y_test_reg = np.array(y_test)

In [ ]:
# x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], 1, 1))
# x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], 1, 1))

In [ ]:
# def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
#   model = Sequential()
#   model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
#   model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
#   model.compile(loss='mean_squared_error', optimizer='adam')
#   return model

In [ ]:
# model = AlphaRNN_(10,0.0)
# model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)

In [ ]:
# model.summary()

In [ ]:
# y_pred_train = model.predict(x_train_reg, verbose = 1)
# MSE_train= mean_squared_error(y_train,y_pred_train)
# r2_train = r2_score(y_train, y_pred_train)

# print("Train MSE: ", MSE_train)
# print("Train r2: ", r2_train)

# y_pred = model.predict(x_test_reg, verbose = 1)
# MSE_test= mean_squared_error(y_test,y_pred)
# r2_test = r2_score(y_test, y_pred)

# print("Test MSE: ", MSE_test)
# print("Test r2: ", r2_test)